In [25]:
import pymorton
from itertools import islice
from copy import deepcopy

mbr = ['2.232','33.34','4.3','34.32']
bb = ['4.0','33.0','4.2','30.32']
inside = True
for j in range(4):
        if not (bb[j]<=mbr[j]):
            inside = False
print(inside)

class node:
    
    def __init__(self, level, leaf=False):
        self.leaf=leaf
        self.level=level
        self.mbr = []
        self.id = []
    def addentry(self,entry):
        id = entry[0]
        mbr = entry[1]
        self.mbr.append(mbr)
        self.id.append(id)

In [26]:
coords_p='coords.txt'
offsets_p='offsets.txt'
polygons = []
mbrs = []
mbr_centres=[]
z_curves=[]
prev_start=0
with open(coords_p) as coordinates:
    with open(offsets_p) as offsets:
        for offset in offsets:
            pointers = [ int(x) for x in offset.split(sep=',') ]
            prev_start=pointers[1]
            polygon = [line.strip().split(sep=',') for line in islice(coordinates,pointers[1]-prev_start, pointers[2]-pointers[1]+1)]
            max_x = max(map(lambda x: x[0], polygon))
            min_x = min(map(lambda x: x[0], polygon))
            max_y = max(map(lambda x: x[1], polygon))
            min_y = min(map(lambda x: x[1], polygon))
            mbrs.append([pointers[0],[min_x,max_x,min_y,max_y]])
            centre = [(float(max_x)+float(min_x))/2,(float(max_y)+float(min_y))/2]
            #mbr_centres.append(centre)
            z_curves.append([pointers[0],pymorton.interleave_latlng(centre[1],centre[0])])
z_curves.sort(key = lambda x: x[1])
mbrs = [mbrs[i] for i in [idx[0] for idx in z_curves]]
del mbr_centres




In [27]:
Tree = []
B = 20
queue = []
lvl = 0
isnonleaf = 0
node_id = 0
for i in range(0,len(mbrs),B):
    #print(i,B+i)
    #node = []
    nodeentries = mbrs[i:i+B]
    queue.append((node_id,nodeentries,lvl))
    Tree.append([isnonleaf,node_id,nodeentries])
    node_id = node_id + 1
    #print(queue[0])
print(len(queue),' nodes at level ',lvl)

500  nodes at level  0


In [28]:
isnonleaf = 1
how_many_nodes = 0
#changelvl = False
reachedRoot=False
node = []
while len(queue)>1:
    i = 0
    if queue[0][2]==2:
        print(queue)
        print(len(queue))
    if len(queue)>1 and len(queue[1][1])<8 and queue[0][2]==queue[1][2]:
        print('size is for ',queue[1][0], 'is', len(queue[1][1]), '<8 redirstributing using',queue[0][0])
        difference = 8 - len(queue[1][1])
        to_be_moved = queue[0][1][-difference:]
        while to_be_moved:
            queue[1][1].insert(0,to_be_moved.pop())
        del queue[0][1][-difference:]
        print('new sizes are', queue[1][0] ,'size:', len(queue[1][1]), queue[0][0],'size:', len(queue[0][1]))






        
    while i<20 and len(queue)>1 and queue[0][2] == lvl: 
        e = queue.pop(0)
        bounding = [x[1] for x in e[1]]
        xs = [item for sublist in [x[:2] for x in bounding] for item in sublist]
        ys = [item for sublist in [x[2:] for x in bounding] for item in sublist]
        max_x = max(xs)
        min_x = min(xs)
        max_y = max(ys)
        min_y = min(ys)
        node.append([e[0],[min_x,max_x,min_y,max_y]])
        i = i + 1
    if queue[0][2]!=lvl:
        #changelvl=True
        if node:
            queue.append((node_id,node,lvl+1))
            Tree.append([isnonleaf,node_id,node])
            node_id = node_id + 1
            how_many_nodes = how_many_nodes + 1
            node = []
        print(how_many_nodes,' nodes at level ', lvl+1)
        lvl = lvl + 1
        how_many_nodes =0
        continue

    if len(queue)==1:
        reachedRoot=True 
        
    if not reachedRoot:
        #print('reached here')
        queue.append((node_id,node,lvl+1))
        Tree.append([isnonleaf,node_id,node])
        how_many_nodes = how_many_nodes + 1
        node_id = node_id + 1
        node = []
    
    if reachedRoot:
        e = queue.pop(0)
        bounding = [x[1] for x in e[1]]
        xs = [item for sublist in [x[:2] for x in bounding] for item in sublist]
        ys = [item for sublist in [x[2:] for x in bounding] for item in sublist]
        max_x = max(xs)
        min_x = min(xs)
        max_y = max(ys)
        min_y = min(ys)
        #print(min_x,max_x,min_y,max_y)
        node.append([e[0],[min_x,max_x,min_y,max_y]])
        Tree.append([isnonleaf,node_id,node])
        how_many_nodes = how_many_nodes + 1
        print(how_many_nodes,' nodes at level ',lvl+1)      


#root = queue.pop(0)        
#Tree.append([isnonleaf,node_id,[root[0],root[1]]])    

25  nodes at level  1
2  nodes at level  2
[(525, [[500, ['-112.606999', '-178.333232', '-14.174098', '44.975665']], [501, ['-112.700998', '-123.734396', '33.752815', '44.936217']], [502, ['-100.003123', '-99.858493', '26.284913', '44.905764']], [503, ['-89.999885', '-98.437586', '29.148613', '33.749474']], [504, ['-101.28464', '-90.522924', '33.42116', '44.96481']], [505, ['-100.005599', '-99.976791', '33.787568', '44.974808']], [506, ['-90.025393', '-95.614557', '33.79594', '39.372375']], [507, ['-100.041896', '-99.867423', '36.579924', '45.000011']], [508, ['-90.035211', '-98.40106', '39.38362', '44.94097']], [509, ['-90.012148', '-95.620265', '40.797663', '44.999972']], [510, ['-118.150933', '-91.51333', '43.624491', '71.307504']], [511, ['-100.058971', '-99.977123', '44.999866', '58.344954']], [512, ['-64.795333', '-95.623846', '17.920394', '48.888283']], [513, ['-84.387166', '-90.002267', '29.13693', '33.74689']], [514, ['-80.603704', '-87.169985', '28.172425', '33.74874']], [515

In [29]:
with open('Rtree.txt','w') as output:
    [output.write(str(item)+'\n') for item in Tree]

ll1 = [0,1,2,3,4]
ll2 = [5,6,7,8]
difference = 3
to_be_moved = ll1[-difference:]
while to_be_moved:
    ll2.insert(0,to_be_moved.pop())
del ll1[-difference:],to_be_moved
print(ll1)
print(ll2)

In [30]:
l1 = [[9,0,1,91],[100,2,89,92]]
xs = [item for sublist in [x[:2] for x in l1] for item in sublist]
ys = [item for sublist in [x[2:] for x in l1] for item in sublist]
print(xs)
print(ys)
max_x = max(xs)
min_x = min(xs)
max_y = max(ys)
min_y = min(ys)
print(max_x,min_x,max_y,min_y)

[9, 0, 100, 2]
[1, 91, 89, 92]
100 0 92 1
